#**Recomendationsystem by KNN** 









In [2]:
import csv
import argparse
import numpy as np
import pandas as pd

from collections import OrderedDict

In [3]:
#read data from movies csv
movie_data = pd.read_csv("movies.csv")
movie_data.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
movie_data.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [7]:
#read data from rating csv
rating_data = pd.read_csv("ratings.csv")
rating_data.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
rating_data.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [11]:
class Recommender():

    #init recommender movie file & rating file
    def __init__(self, movie_filename, rating_filename):

        # read movie file and create dictionary _movie_names
        self._movie_names = {}
        with open(movie_filename, 'r', encoding = 'utf8') as csv_reader:
            reader = csv.reader(csv_reader)
            next(reader, None)

            for line in reader:
                movieid = line[0]
                moviename = line[1]
                # ignore line[2], genre
                self._movie_names[movieid] = moviename

        # read rating file and create _movie_ratings (ratings for a movie)
        # and _user_ratings (ratings by a user) dicts
        self._movie_ratings = {}
        self._user_ratings = {}

        with open(rating_filename, 'r', encoding = 'utf8') as csv_reader:
            reader = csv.reader(csv_reader)
            next(reader, None)

            for line in reader:

                userid = line[0]
                movieid = line[1]
                rating = line[2]
                # ignore line[3], timestamp
                if userid in self._user_ratings:
                    userrats = self._user_ratings[userid]
                else:
                    userrats = {}
                userrats.update({movieid: float(rating)})
                self._user_ratings[userid] = userrats

                if movieid in self._movie_ratings:
                    movierats = self._movie_ratings[movieid]
                else:
                    movierats = {}
                movierats.update({userid: float(rating)})
                self._movie_ratings[movieid] = movierats


    ####
    # USER TO USER
    ####

    #Hàm tính toán sự giống nhau giữa những người dùng bằng rating của người dùng
    def similarity_between_users(self, ratings1, ratings2):
        ''' PARAMETERS
        - ratings1, ratings2: 2 biến đại diện cho danh sách xếp hạng của hai người dùng
        ----------
        RETURNS
        - biến kiểu float đại diện cho sự giống nhau
        '''
        avg1 = np.array(list(ratings1.values())).mean() # giá trị trung bình rating của người dùng 1
        avg2 = np.array(list(ratings2.values())).mean() # giá trị trung bình rating của người dùng 2

        #bộ giao giữa rating1 & rating2
        S = set(ratings1.keys()).intersection(set(ratings2.keys()))

        num, den1, den2 = 0., 0., 0.
        # Sử dụng Công thức tính toán độ tương tự theo hệ số tương quan Pearson
        for movieid in S:
            num += (ratings1[movieid] - avg1)*(ratings2[movieid] - avg2)
            den1 += (ratings1[movieid] - avg1)**2
            den2 += (ratings2[movieid] - avg2)**2

        # return round(num / (den1*den2), 5) if (den1 != 0 and den2 != 0) else 0
        return num / np.sqrt(den1*den2) if (den1 != 0 and den2 != 0) else 0

    # Chức năng dự đoán xếp hạng của một bộ phim cho một người dùng nhất định
    def predict_rating(self, movie, neighbours):
        ''' Function that predicts the rating of a movie for a given user
        ----------
        PARAMETERS
        - movie: movieId của bộ phim muốn dự đoán xếp hạng
        - neighbours: danh sách 'knn' hàng xóm gần nhất với người dùng được đề cập
        ----------
        RETURNS
        - kiểu dl float giữa 1-5 biểu thị xếp hạng cho bộ phim cụ thể đó
        '''
        num, den = 0., 0.
        for neighbour, similarity in neighbours.items():
            # Kiểm tra xem phim đã được người dùng cụ thể này xem chưa
            if movie in self._user_ratings[neighbour]:
                # xem xét những điểm tương đồng tích cực đáng kể
                if similarity >= .01:
                    num += similarity * (self._user_ratings[neighbour][movie])
                    den += similarity

        return num/den if (den != 0) else 0

    # Hàm trả về những bộ phim 'k' có nhiều khả năng nhất để một người dùng cụ thể thích
    def recommend_user_to_user(self, rating_list, knn = 50, k = 10):
        ''' 
        PARAMETERS
        - rating_list: biến biểu thị danh sách xếp hạng cho người dùng mới
        - knn: integer đại diện cho số láng giềng gần nhất cần tính đến
        - k: integer đại diện cho số lượng đề xuất cần nhận
        ----------
        RETURNS
        - biến 'k' các bộ phim được đề xuất cao nhất để xem cho người dùng
        '''
        neighbours = {}
        for user, ratings in self._user_ratings.items():
            neighbours[user] = self.similarity_between_users(rating_list, ratings)
        # Sắp xếp độ giảm dần tương tự
        neighbours = sorted(neighbours.items(), key = lambda x: -x[1])
        # Gắn với những người dùng 'knn' gần nhất
        neighbours = OrderedDict(neighbours[:knn])

        # Nhận những bộ phim đã được đánh giá bởi một số knn hàng xóm và người dùng chưa đánh giá.
        movies_to_rate = set()
        for user in neighbours.keys():
            movies_to_rate = movies_to_rate.union(set(self._user_ratings[user].keys()))
        movies_to_rate = [x for x in movies_to_rate if x not in rating_list.keys()]

        # Dự đoán xếp hạng mà người dùng của chúng tôi sẽ xếp hạng cho mỗi phim trong 'movies_to_rate'
        pred = {}
        for movie in movies_to_rate:
            pred[movie] = self.predict_rating(movie, neighbours)
        # Sắp xếp 
        pred = sorted(pred.items(), key = lambda x: -x[1])
        # Gắn với knn gần nhất
        pred = OrderedDict(pred[:k])

        return pred


    ####
    # ITEM TO ITEM
    ####
    #Hàm tính toán sự giống nhau giữa các mục (phim)
    def similarity_between_items(self, ratings1, ratings2):
        ''' 
        PARAMETERS
        - ratings1, ratings2: 2 biến đại diện cho danh sách xếp hạng của hai người dùng
        ----------
        RETURNS
        - float biến đại diện cho sự giống nhau
        '''
        avg1 = np.array(list(ratings1.values())).mean()
        avg2 = np.array(list(ratings2.values())).mean()

        S = set(ratings1.keys()).intersection(set(ratings2.keys()))

        num, den1, den2 = 0., 0., 0.
        for userid in S:
            num += (ratings1[userid] - avg1)*(ratings2[userid] - avg2)
            den1 += (ratings1[userid] - avg1)**2
            den2 += (ratings2[userid] - avg2)**2

        return num / np.sqrt(den1*den2) if (den1 != 0 and den2 != 0) else 0

    #Chức năng dự đoán xếp hạng của một bộ phim cho một người dùng nhất định
    def predict_rating2(self, rating_list, neighbours):
        ''' 
        ----------
        PARAMETERS
        - rating_list: biến đại diện cho danh sách xếp hạng của người dùng cụ thể
        - neighbours: danh sách 'knn' hàng xóm gần nhất với người dùng được đề cập
        ----------
        RETURNS
        - kiểu dl float giữa 1-5 biểu thị xếp hạng cho bộ phim cụ thể đó
        '''
        num, den = 0., 0.
        for movie, similarity in neighbours.items():
            # We only want to consider significant positive similarities
            if similarity >= .01:
                num += similarity * rating_list[movie]
                den += similarity

        return num / den if (den != 0) else 0

    #Hàm trả về những bộ phim 'k' có nhiều khả năng nhất để một người dùng cụ thể thích
    def recommend_item_to_item(self, rating_list, knn, k):
        '''  
        PARAMETERS
        - rating_list: biến biểu thị danh sách xếp hạng cho người dùng mới
        - knn: integer đại diện cho số láng giềng gần nhất cần tính đến
        - k: integer đại diện cho số lượng đề xuất cần nhận
        ----------
        RETURNS
        - biến 'k' các bộ phim được đề xuất cao nhất để xem cho người dùng
        '''
        # Nhận những bộ phim mà người dùng chưa đánh giá
        movies_to_rate = [x for x in self._movie_ratings.keys() if x not in rating_list.keys()]

        # Nhận những bộ phim gần nhất
        neighbours = {}
        for movie1 in movies_to_rate:
            ratings = self._movie_ratings[movie1]
            neighbours[movie1] = {}
            for movie2 in rating_list.keys():
                neighbours[movie1][movie2] = self.similarity_between_items(ratings, self._movie_ratings[movie2])

        # Sắp xếp các biến và chọn knn xuất hiện đầu tiên
        for movie in neighbours:
            neighbours[movie] = sorted(neighbours[movie].items(), key = lambda x: -x[1])
            neighbours[movie] = OrderedDict(neighbours[movie][:knn])
        
        # Dự đoán xếp hạng cho mọi bộ phim có thể có
        pred = {}
        for movie in neighbours:
            pred[movie] = self.predict_rating2(rating_list, neighbours[movie])
        #Sắp xếp các bộ phim theo xếp hạng và gắn với chữ k đầu tiên
        pred = sorted(pred.items(), key = lambda x: -x[1])
        pred = OrderedDict(pred[:k])

        return pred


In [ ]:
if __name__ == '__main__':
    # Phân tích cú pháp các đối số
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '-knn', default = 50, type = int, help = 'Number of closest neighbours to consider.'
    )
    parser.add_argument(
        '-k', default = 10, type = int, help = 'Number of objects shown to the user.'
    )
    # Lấy các đối số
    args, unknown = parser.parse_known_args()
    knn = args.knn
    k = args.k

    # Tạo lớp đọc các tệp
    r = Recommender("movies.csv","ratings.csv")

    # Liên tục, yêu cầu danh sách các phim và xếp hạng, 
    # đồng thời yêu cầu Người giới thiệu cung cấp các đề xuất cho danh sách này và in tiêu đề của các phim được đề xuất và xếp hạng dự đoán của chúng:
    while (input('New list of movie ratings? [y/n] : ') == 'y'):
        print('\nEnd the list by typing 0 in the movieID\n')
        rating_list = {}
        movie = input('MovieID : ')
        while (movie != '0'):
            assert (float(movie) <= 193609) & (float(movie) >= 1), f"MovieID {movie} doesn't exist. Ensure it is in range(1,193609)"
            # Input a rating and ensure it is valid
            rating = float(input('Rating : '))
            assert (rating <= 5.) & (rating >= .5), 'Rating range must be between 1 and 5'
            # Append the rating to the rating list and ask for a new movieID
            rating_list[movie] = rating
            movie = input('MovieID : ')
        print()

        # Dự đoán xếp hạng bằng cách sử dụng giới thiệu Người dùng với Người dùng
        print('-' * 60)
        print('Using User-to-User recommendation:')
        recommended = r.recommend_user_to_user(rating_list, knn, k)
        for movieid, rate in recommended.items():
            print(f" - {r._movie_names[movieid]} : {rate}")
        print()
        #Dự đoán xếp hạng bằng cách sử dụng giới thiệu Item với Item
        print('-' * 60)
        print('using Item-to-Item recommendation:')
        recommended = r.recommend_item_to_item(rating_list, knn, k)
        for movieid, rate in recommended.items():
            print(f" - {r._movie_names[movieid]} : {rate}")
        print()

New list of movie ratings? [y/n] : y

End the list by typing 0 in the movieID

MovieID : 656
